In [54]:
import numpy as np
import cv2
from minio import Minio
from camera_intrinsic import utils as camera_utils

In [73]:
data1 = np.load("/calibration/ros2/src/hand_eye/hand_eye/2024-07-21_16_38_38.100940.npy", allow_pickle=True)
data2 = np.load("/calibration/ros2/src/hand_eye/hand_eye/2024-07-21_16_49_35.921496.npy", allow_pickle=True)
data  = np.vstack((data1, data2))
print(data.shape)
cam_data = np.load("/calibration/ros2/src/hand_eye/hand_eye/intrinsic_2024-07-07_14-56-53_intrinsics (1).npz", allow_pickle=True)
frames, poses = data[:, 0], data[:, 1]

(2129, 2)


In [74]:
cam_data["dist"]

array([[ 3.10334662e-02,  6.15702392e-01,  8.31168943e-04,
        -3.26508230e-04, -2.21216690e+00]])

In [75]:
for frame in frames:
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for 'ESC'
        break
cv2.destroyAllWindows()

In [79]:
# Get sharp frames, blurried frames are less likely to have accurate corners
sharp_frames_idxs = camera_utils.get_sharp_frames(frames, 280)
sharp_poses, sharp_frames  = poses[sharp_frames_idxs], frames[sharp_frames_idxs]

# Only process sufficiently different frames to reduce computation time and avoid overfitting
representative_frames_idxs = camera_utils.get_n_representative_frames(frames[sharp_frames_idxs], num_frames=60)

representative_poses, representative_frames  = sharp_poses[representative_frames_idxs], sharp_frames[representative_frames_idxs]

Selecting sharp frames out of 2129 frames
Found 551 sharp frames out of 2129 with threshold 280
Selecting 60 representative frames out of 551 frames
Reduced frames from 110592 to 57 dimensions keeping 95.0 of variance
Selected 60 representative frames out of 276 frames


In [80]:
for frame in representative_frames:
    cv2.imshow("frame", frame)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [81]:
chessboard_pts = camera_utils.get_chessboard_pts((9, 6))

R_gripper2base_r = []
t_gripper2base_r = []
R_target2cam_r   = []
t_target2cam_r   = []

for frame, pose in zip(representative_frames, representative_poses):
    ret, corners = camera_utils.find_chessboard_corners(frame)    
    if ret:
        ret2, rvec, tvec = cv2.solvePnP(chessboard_pts, corners, cam_data["mtx"], cam_data["dist"])
        if ret2:
            R_gripper2base_r.append(pose[:3, :3])
            t_gripper2base_r.append(pose[:3, 3].reshape((3, 1)))
            
            rot ,jacobian = cv2.Rodrigues(rvec)
            R_target2cam_r.append(rot)
            t_target2cam_r.append(tvec)

R_cam2gripper, t_cam2gripper = cv2.calibrateHandEye(
    R_gripper2base_r, 
    t_gripper2base_r, 
    R_target2cam_r, 
    t_target2cam_r, 
    method = cv2.CALIB_HAND_EYE_TSAI
)
print(R_cam2gripper, t_cam2gripper)

Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboard corners in image
Found chessboa

In [82]:
np.savez("hand_eye", r_cam2gripper = R_cam2gripper, t_cam2gripper = t_cam2gripper)